In [143]:
import numpy as np # linear algebra
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
from imutils.contours import sort_contours
import imutils
import asciitomathml.asciitomathml
import re

from IPython.core.debugger import set_trace
from IPython.core.display import display, HTML


In [144]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [145]:
train_datagen = ImageDataGenerator(
    rescale = 1./255, 
    shear_range = 0.2, 
    zoom_range = 0.2,
    validation_split = 0.25
)

data_path='extracted_images'
train_set = train_datagen.flow_from_directory(
    data_path, 
    target_size = (45, 45), 
    color_mode = 'grayscale',
    batch_size = 32,
    class_mode = 'categorical',
    shuffle = True,
    subset='training',
    seed = 123
)
valid_set = train_datagen.flow_from_directory(
    data_path, 
    target_size = (45, 45), 
    color_mode = 'grayscale',
    batch_size = 32,
    class_mode = 'categorical',
    shuffle = True,
    subset='validation',
    seed = 123
)

Found 282007 images belonging to 82 classes.
Found 93967 images belonging to 82 classes.


In [146]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(45, 45, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(82 , activation='softmax'))
# compile model
adam = tf.keras.optimizers.Adam(lr = 5e-4)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

D:\Anaconda\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [ ]:
model.fit_generator(train_set,validation_data=valid_set,epochs=1, verbose=1)

D:\Anaconda\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


8490/8813 [===========================>..] - ETA: 4:46 - loss: 0.8251 - accuracy: 0.7836

In [ ]:
val_loss, val_accuracy = model.evaluate(valid_set)
print(val_loss,val_accuracy)

In [ ]:
def prediction(img):
    #img = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
    plt.imshow(img, cmap = 'gray')
    img = cv2.resize(img,(45, 45))
    norm_image = cv2.normalize(img, None, alpha = 0, beta = 1, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)
    #norm_image=img/255
    norm_image = norm_image.reshape((norm_image.shape[0], norm_image.shape[1], 1))
    case = np.asarray([norm_image])
    pred = model.predict_classes([case])
    
    return ([i for i in train_set.class_indices if train_set.class_indices[i]==(pred[0])][0],pred)

In [ ]:
label_map = (train_set.class_indices)
label_map

In [ ]:
image = cv2.imread('test_image2.png')
plt.imshow(image)

In [ ]:
image = cv2.imread('test_image.jpeg')
#image = cv2.resize(image,(300,300))
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
# perform edge detection, find contours in the edge map, and sort the
# resulting contours from left-to-right
edged = cv2.Canny(blurred, 30, 150)
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sort_contours(cnts, method="left-to-right")[0]
chars=[]
for c in cnts:
    # compute the bounding box of the contour
    (x, y, w, h) = cv2.boundingRect(c)
    # filter out bounding boxes, ensuring they are neither too small
    # nor too large
    if w*h>1200:
        
        roi = gray[y:y + h, x:x + w]
        chars.append(prediction(roi))
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(image)

In [ ]:
chars

In [ ]:
labels=[i for i in train_set.class_indices]
print(labels)

In [ ]:
eq=[]
for i in ((chars)):
    eq.append(i[0])

In [ ]:
asciiArr=[]
for i in range(0, len(eq)):
    if re.match('[a-zA-Z]', eq[i]):
        asciiArr.append(eq[i])
    elif (re.match('[0-9]', eq[i])) and (re.match('[a-zA-Z]', eq[i-1])):
        asciiArr.append('^')
        asciiArr.append(eq[i])
    else:
        asciiArr.append(eq[i])

In [ ]:
listToStr = ' '.join(map(str, asciiArr))
math_obj =  asciitomathml.asciitomathml.AsciiMathML(mstyle={'displaystyle':'true'})
math_obj.parse_string(listToStr)

In [ ]:
xml_string = math_obj.to_xml_string(encoding='unicode') 

In [ ]:
display(HTML(xml_string))
